In [8]:
import sys
sys.path.append('../src')
sys.path.append('..')

from main import generate_refined_data

from tqdm import tqdm
import pandas as pd

In [9]:
with open('../src/init.py') as f:
    code = compile(f.read(), '../src/init.py', 'exec')
    exec(code)

100%|██████████| 899/899 [00:00<00:00, 983.32it/s] 


In [10]:
df = pd.read_csv('../data/data.csv')
df = df.sort_values(by='timestamp', ascending=True)
df = df.reset_index(drop=True)
df.head()

,timestamp,mac,map_id,rssi,site_id,type,x,y
0,1.723194e+09,6e55c33d4bf94fc912546aa53df0a83e6cd97bc542e861...,7445f317-78d8-4a93-add1-ac223c538c55,-55.300000,48b01af6-138b-465d-8996-bace824f5726,wifi,78.165060,49.166220
1,1.723194e+09,211dcd244bd609da55415aa490f81a19fda9563ec32425...,527aaffb-bc14-4013-b03c-d81bfde894bf,-62.000000,48b01af6-138b-465d-8996-bace824f5726,wifi,48.303600,22.940083
2,1.723194e+09,c806634ea9528565a397a6d8e2c6c11733892ed6c683d6...,529cbd22-3119-42b3-9bf9-086d108ed159,-47.550003,48b01af6-138b-465d-8996-bace824f5726,wifi,15.859675,18.266554
3,1.723194e+09,bf07a6f97d267a8841751725f1c270399910b53058b918...,162b807a-8c8b-4169-8c33-91882455b0a8,-68.200000,48b01af6-138b-465d-8996-bace824f5726,wifi,67.894010,37.696960
4,1.723194e+09,074b3881ba8aa8155093cf20719051392f85885c03bf8b...,caa07ab5-c4fb-4848-a8bf-57b482b429e6,-41.850000,48b01af6-138b-465d-8996-bace824f5726,wifi,52.415646,34.921520


In [11]:
# split into 5 minute intervals
# feed batches into main script -> generates refined data
# run script to turn refined data into occupancy stats
# run script to turn occupancy stats into density maps

In [12]:
timestamps = df['timestamp'].to_numpy()
cutoff_indices = [0]

prev_timestamp = timestamps[0]
for i, timestamp in enumerate(timestamps):

    if timestamp - prev_timestamp > 60*5:
        cutoff_indices.append(i)
        prev_timestamp = timestamp

    if len(cutoff_indices) == 2*6*24*3:
        break

print(cutoff_indices)

[0, 2026, 4659, 8703, 13386, 18911, 25192, 31964, 39636, 47386, 55804, 65327, 75290, 84768, 94282, 104142, 114803, 125615, 135843, 146291, 156441, 167336, 178184, 190182, 202459, 213763, 225234, 236686, 248250, 260796, 272795, 284573, 296370, 306690, 316883, 326793, 334762, 342985, 349319, 356786, 363717, 371538, 378697, 386281, 393409, 400721, 407162, 412832, 418414, 424243, 429258, 434014, 439196, 444140, 449621, 454811, 460450, 466188, 471152, 477215, 482019, 487248, 492569, 497481, 502732, 507914, 513612, 518892, 523859, 529061, 534747, 539526, 544553, 550185, 555853, 561721, 567677, 573773, 578204, 583179, 587789, 591702, 596689, 600642, 605025, 608653, 612927, 616992, 620838, 624473, 628115, 631885, 634861, 637987, 641103, 644236, 646834, 649562, 652494, 654698, 657125, 659604, 661932, 663965, 665946, 668167, 670089, 672646, 674981, 676854, 678835, 680685, 682636, 684738, 686487, 688299, 689937, 691355, 692966, 694481, 696089, 697796, 699452, 700831, 702245, 703760, 705386, 70686

In [13]:
batches = []

for i in tqdm(range(len(cutoff_indices))):
    if i == len(cutoff_indices) - 1:
        break

    batch = df.iloc[cutoff_indices[i]:cutoff_indices[i+1]]
    batches.append(batch)

100%|█████████▉| 863/864 [00:00<00:00, 11069.20it/s]


In [14]:
print(len(batches))
print(len(batches[3]))

863
4683


In [15]:
for i, batch in enumerate(tqdm(batches)):
    if i == 0:
        generate_refined_data(batch, first_batch=True)
    else:
        generate_refined_data(batch)

  0%|          | 0/863 [00:00<?, ?it/s]

  0%|          | 1/863 [00:00<09:08,  1.57it/s]/Users/carl/Desktop/ict/wifi-tracking2/test/../src/device.py:66: RuntimeWarning: invalid value encountered in scalar divide
  z = radius_to_prev / prev_error
  0%|          | 2/863 [00:01<06:56,  2.07it/s]/Users/carl/Desktop/ict/wifi-tracking2/test/../src/device.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  z = radius_to_prev / prev_error
 55%|█████▍    | 471/863 [31:22<26:06,  4.00s/it]  


KeyboardInterrupt: 